# Creation of the Document Repository (Wikipedia)

## 1 Identification of the corpus from DBPedia

### 1.1 Get DBPedia concepts from the root concept

In [10]:
#############################################
# Creating tables:
# Unique pages:
# Table of cleaning pages:
#############################################

from bd_vr import BDdatos

db = BDdatos()
con = db.conectar()

tableDB = 'DBCSubconceptsVR'  # Table with subconcepts 

cur = con.cursor()

# Creating table:
cur.execute(u"""CREATE TABLE """ + tableDB + """ (`date` VARCHAR(12) DEFAULT NULL,
  `r` TEXT CHARACTER SET utf8 COLLATE utf8_general_ci,
  `c1` TEXT CHARACTER SET utf8 COLLATE utf8_general_ci,
  `c2` TEXT CHARACTER SET utf8 COLLATE utf8_general_ci,
  `c3` TEXT,
  `level` INT DEFAULT NULL);""")

db.saveDB(con)
db.closeDB(con)

In [11]:
from SPARQLWrapper import SPARQLWrapper, JSON, RDF, XML
from bd_vr import BDdatos
import codecs
import json
from datetime import date

db = BDdatos()
con = db.conectar()

# Variables to start the query process. To change de rootConcept.

rootConcept = ["http://dbpedia.org/resource/Category:Virtual_reality"] # set the root concept
e = "http://live.dbpedia.org/sparql"  # set the EndPoint: DBPedia Live


def getSubConcepts():
    """
        Get sub-soncepts of the root concept(s). Retrieve subconcepts located in three hops.
    """
    
    varP = ['c1', 'c2', 'c3', 'level']
    sparql = SPARQLWrapper(e)
    
    query = """select distinct ?level ?r ?c1 ?c2 ?c3
    {
        {VALUES (?r ?level) {(<%s> 1)}
            ?r ^skos:broader ?c1.}
        union
        {VALUES (?r ?level) {(<%s> 2)}
            ?r ^skos:broader ?c1. ?c1 ^skos:broader ?c2.}
        union
        {VALUES (?r ?level) {(<%s> 3)}
            ?r ^skos:broader ?c3. ?c3 ^skos:broader ?c2. ?c2 ^skos:broader ?c1.}
    }
    """
    numSC = 0 # number of subconcepts
    print("\t Processing...")
    for res in rootConcept:
        print("Root concept:", res)  
        sparql.setQuery(query%(res, res, res))
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
        for result in results["results"]["bindings"]:
            numSC += 1
            var1 = var2 = var3 = var4 = ''
            if varP[0] in result:
                var1 = result[varP[0]]["value"]
            if varP[1] in result:
                var2 = result[varP[1]]["value"]
            if varP[2] in result:
                var3 = result[varP[2]]["value"]
            if varP[3] in result:
                var4 = result[varP[3]]["value"]
            db.DBCSubconceptsVR(db, date.today().strftime("%d/%m/%Y"), res, var1, var2, var3, var4, tableDB)
    
    print("Number of results:", numSC)
    
    
# To RUN:
getSubConcepts()  # Get subconcepts


	 Processing...
Root concept: http://dbpedia.org/resource/Category:Virtual_reality
Number of results: 772


### 1.2 Getting a unique list of concepts:

In [12]:
from bd_vr import BDdatos

db = BDdatos()
con = db.conectar()

print("...Getting unique concepts")

tableDB = 'DBCSubconceptsVR'
tableDBlist = 'DBCSubconceptsVRList'
tableDBIDs = 'DBCSubconceptsVRIDs'

db.DBCGetUniqueConcepts(db, tableDB, tableDBlist, tableDBIDs)

db.saveDB(con)
db.closeDB(con)


...Getting unique concepts


### 1.3. Getting Wikipage links and their metadata from DBPedia Live:

<b>a. Creating the table:</b>

In [13]:
from SPARQLWrapper import SPARQLWrapper, JSON, RDF, XML
from datetime import date
from bd_vr import BDdatos

db = BDdatos()
con = db.conectar()

e = "http://live.dbpedia.org/sparql" 
otable = 'DBSubconcepts_WikiPages'  # table to fill with wikipages data

cur = con.cursor()
# Create table:
cur.execute(u"""CREATE TABLE """ + otable + """ (`date` date, `concept` text, """ +
                   """ `dbrResource` text, `wikiPage` text, `wikiPageModified` date DEFAULT NULL, `max_outDegree` INT DEFAULT NULL) """ +
                   """ ENGINE=InnoDB DEFAULT CHARSET=utf8;""")

db.saveDB(con)
db.closeDB(con)


<b> b. Querying data: </b>

In [14]:
con = db.conectar()


###########
# Get Pages:

def getWikiPagesFromDBConcepts(stable, otable, l1, l2):
    cur = con.cursor()
    print("Getting links from DBpedia:")

    # Insert data into new table:
    cur.execute("""select distinct id, c from """ + stable + """ where id >""" +str(l1)+' and id <= ' + str(l2))
    rowList = cur.fetchall()
    for row in rowList:
        idC = row[0]
        cat = row[1]
        print(idC, cat)
        results = getMetadata(cat)
        if results:
            for result in results:
                res = page = max_modif  = ''
                max_outDegree = 0
                res = result['r']["value"]
                if 'primaryTopic' in result:
                    page = result['primaryTopic']["value"]
                if 'max_modif' in result:
                    max_modif = result['max_modif']["value"]
                if 'max_outDegree' in result:
                    max_outDegree = result['max_outDegree']["value"]
            
                db.DBSubconcepts_WikiPages(db, date.today().strftime("%Y-%m-%d"), cat, res, page, max_modif, max_outDegree, otable)

def getMetadata(cat):
    sparql = SPARQLWrapper(e)
    # Pendiente el type de cada recurso:
    q = """SELECT DISTINCT ?r ?primaryTopic (substr(str(xsd:date(max(?modif))), 1, 10) AS ?max_modif) 
    (str(max(?outDegree)) AS ?max_outDegree)
    WHERE{
    VALUES ?c {<%s>}
    ?r dct:subject ?c .
    OPTIONAL {?r dbo:wikiPageModified ?modif.}
    OPTIONAL {?r dbo:wikiPageOutDegree ?outDegree.}
    OPTIONAL {?r foaf:isPrimaryTopicOf ?primaryTopic.}
    }"""
    sparql.setQuery(q%(cat))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results["results"]["bindings"]

#########################
# Manual check after run

# Send range of ids to get pages: 281 (max id from tableDBIDs = DBCSubconceptsCOVIDIDs)
getWikiPagesFromDBConcepts(tableDBIDs, otable, 0, 282) 



Getting links from DBpedia:
1 http://dbpedia.org/resource/Category:Virtual_reality_organizations
2 http://dbpedia.org/resource/Category:Virtual_reality_communities
3 http://dbpedia.org/resource/Category:Holography
4 http://dbpedia.org/resource/Category:Virtual_reality_games
5 http://dbpedia.org/resource/Category:Virtual_reality_works
6 http://dbpedia.org/resource/Category:Virtual_reality_in_fiction
7 http://dbpedia.org/resource/Category:Haptic_technology
8 http://dbpedia.org/resource/Category:Ray_tracing_(graphics)
9 http://dbpedia.org/resource/Category:Psychoacoustics
10 http://dbpedia.org/resource/Category:Mixed_reality
11 http://dbpedia.org/resource/Category:Virtual_reality_pioneers
12 http://dbpedia.org/resource/Category:Virtual_avatars
13 http://dbpedia.org/resource/Category:Virtual_economy
14 http://dbpedia.org/resource/Category:Virtual_reality_headsets
15 http://dbpedia.org/resource/Category:Video_gaming
16 http://dbpedia.org/resource/Category:DVD_interactive_technology
17 http:

133 http://dbpedia.org/resource/Category:Nintendo_hardware
134 http://dbpedia.org/resource/Category:Japan-only_video_game_hardware
135 http://dbpedia.org/resource/Category:Video_game_accessories
136 http://dbpedia.org/resource/Category:Video_game_platforms
137 http://dbpedia.org/resource/Category:Modchips
138 http://dbpedia.org/resource/Category:Microsoft_video_game_hardware
139 http://dbpedia.org/resource/Category:Video_games_by_theme
140 http://dbpedia.org/resource/Category:History_of_video_games
141 http://dbpedia.org/resource/Category:Advergames
142 http://dbpedia.org/resource/Category:Video_games_by_genre
143 http://dbpedia.org/resource/Category:Video_games_by_game_engine
144 http://dbpedia.org/resource/Category:Video_gaming_operating_systems
145 http://dbpedia.org/resource/Category:Video_games_adapted_into_novels
146 http://dbpedia.org/resource/Category:Video_games_by_company
147 http://dbpedia.org/resource/Category:Video_games_by_development_status
148 http://dbpedia.org/resourc

DataError: 1292 (22007): Incorrect date value: '' for column 'wikiPageModified' at row 1

<b> c. Validating Wikipage links (there are some redirects)</b>

In [15]:
#############################################
# Creating tables:
# Unique pages:
# Table of cleaning pages:
#############################################

from SPARQLWrapper import SPARQLWrapper, JSON, RDF, XML
from datetime import date
from bd_vr import BDdatos

db = BDdatos()
con = db.conectar()


ttable = 'WikiPagesIDs_tmp'  # Temporal table with unique wikipage links retrieved from DBPedia
rtable = 'WikiPagesRedirects'

cur = con.cursor()
# Create table:
cur.execute(u"""CREATE TABLE """ + ttable + """ (id INT AUTO_INCREMENT PRIMARY KEY)"""+
            """SELECT DISTINCT wikipage page, count(*) AS freq """ +
            """FROM """ + otable + """ GROUP BY page HAVING length(wikipage) > 0;
""")

cur.execute(u"""CREATE TABLE `""" + rtable + """` (`date` DATE, id INT, """+ 
            """`dbpUrl` TEXT, `returnedUrl` TEXT);""")

db.saveDB(con)
db.closeDB(con)

In [16]:
#############################################
# Getting redirects from Wikipedia
#############################################

from datetime import date
from bd_vr import BDdatos
import codecs
import json
import re 
from bs4 import BeautifulSoup 

db = BDdatos()
con = db.conectar()


URL = "https://en.wikipedia.org/w/api.php"
import requests
S = requests.Session()

################    
# Get Redirects:
def getRedirects(l1, l2):
    cur = con.cursor()
    print("Getting redirects:")
    cur = con.cursor()
    cur.execute("""SELECT distinct id, page FROM """ + ttable +  """ WHERE id >""" +str(l1)+' and id <= ' + str(l2))
    rowList = cur.fetchall()
    for row in rowList:
        idP = row[0]
        dbpediaUrl = row[1]
        print(idP, dbpediaUrl)
        returnedUrl = getLink(dbpediaUrl).replace('https://', 'http://')
        db.WikiPagesRedirects(db, date.today().strftime("%Y-%m-%d"), idP, dbpediaUrl, returnedUrl, rtable)

def getLink(page):
    """Check if it is a redirect"""
    r = requests.get(page)
    c = r.content
    soupC = BeautifulSoup(c,"html.parser")
    link=soupC.find('link',{'rel':'canonical'})['href']
    return link


#########################
# Manual check after run
# To run: urls (getting from DBPedia) to clean
getRedirects(0, 3410) # The second parameter is the max id in ttable = WikiPagesIDs_tmp



Getting redirects:
1 http://en.wikipedia.org/wiki/National_Film_Board_of_Canada
2 http://en.wikipedia.org/wiki/Animaze_Montreal_International_Film_Festival
3 http://en.wikipedia.org/wiki/Virtual_Reality_Applications_Center
4 http://en.wikipedia.org/wiki/VRAC
5 http://en.wikipedia.org/wiki/International_Stereoscopic_Union
6 http://en.wikipedia.org/wiki/Virtual_Reality_and_Education_Laboratory
7 http://en.wikipedia.org/wiki/Immerse_Learning
8 http://en.wikipedia.org/wiki/DiSTI
9 http://en.wikipedia.org/wiki/Global_Virtual_Reality_Association
10 http://en.wikipedia.org/wiki/Multiverse_Foundation
11 http://en.wikipedia.org/wiki/Blue_Brain_Project
12 http://en.wikipedia.org/wiki/Web3D_Consortium
13 http://en.wikipedia.org/wiki/VRTO
14 http://en.wikipedia.org/wiki/IBM_Virtual_Universe_Community
15 http://en.wikipedia.org/wiki/Festival_of_International_Virtual_&_Augmented_Reality_Stories
16 http://en.wikipedia.org/wiki/Contact_consortium
17 http://en.wikipedia.org/wiki/Google_Expeditions
18 h

165 http://en.wikipedia.org/wiki/Cal_Ripken's_Real_Baseball
166 http://en.wikipedia.org/wiki/Space_Shuttle_Mission_2007
167 http://en.wikipedia.org/wiki/Survivalcraft
168 http://en.wikipedia.org/wiki/Boneworks
169 http://en.wikipedia.org/wiki/Dreams_(video_game)
170 http://en.wikipedia.org/wiki/Team_Fortress_2
171 http://en.wikipedia.org/wiki/Rangi_(video_game)
172 http://en.wikipedia.org/wiki/Space_Pirate_Trainer
173 http://en.wikipedia.org/wiki/Trover_Saves_the_Universe
174 http://en.wikipedia.org/wiki/Starship_Commander:_Arcade
175 http://en.wikipedia.org/wiki/Minecraft
176 http://en.wikipedia.org/wiki/Missile_Command_3D
177 http://en.wikipedia.org/wiki/Screen_sports
178 http://en.wikipedia.org/wiki/Asgard's_Wrath
179 http://en.wikipedia.org/wiki/Chronos_(2016_video_game)
180 http://en.wikipedia.org/wiki/Pavlov_VR
181 http://en.wikipedia.org/wiki/Please,_Don't_Touch_Anything
182 http://en.wikipedia.org/wiki/Budget_Cuts
183 http://en.wikipedia.org/wiki/Project_Wingman
184 http://en.w

322 http://en.wikipedia.org/wiki/Novint_Technologies
323 http://en.wikipedia.org/wiki/Haptic_suit
324 http://en.wikipedia.org/wiki/BlackBerry_Storm
325 http://en.wikipedia.org/wiki/Woojer
326 http://en.wikipedia.org/wiki/Tactile_technology
327 http://en.wikipedia.org/wiki/Refreshable_braille_display
328 http://en.wikipedia.org/wiki/Sixaxis
329 http://en.wikipedia.org/wiki/BlackBerry_Storm_2
330 http://en.wikipedia.org/wiki/Battlefield_V
331 http://en.wikipedia.org/wiki/DirectX_Raytracing
332 http://en.wikipedia.org/wiki/List_of_ray_tracing_software
333 http://en.wikipedia.org/wiki/Rayshade
334 http://en.wikipedia.org/wiki/Wolfenstein:_Ray_Traced
335 http://en.wikipedia.org/wiki/OptiX
336 http://en.wikipedia.org/wiki/Quake_Wars:_Ray_Traced
337 http://en.wikipedia.org/wiki/Ray-tracing_hardware
338 http://en.wikipedia.org/wiki/Progressive_refinement
339 http://en.wikipedia.org/wiki/Distributed_ray_tracing
340 http://en.wikipedia.org/wiki/OpenRL
341 http://en.wikipedia.org/wiki/Ray_tracing

488 http://en.wikipedia.org/wiki/Oculus_Quest
489 http://en.wikipedia.org/wiki/Z800_3DVisor
490 http://en.wikipedia.org/wiki/Google_Daydream
491 http://en.wikipedia.org/wiki/Oculus_Rift_CV1
492 http://en.wikipedia.org/wiki/Virtual_reality_headset
493 http://en.wikipedia.org/wiki/PlayStation_VR
494 http://en.wikipedia.org/wiki/Comparison_of_virtual_reality_headsets
495 http://en.wikipedia.org/wiki/Nokia_OZO
496 http://en.wikipedia.org/wiki/VirtualLink
497 http://en.wikipedia.org/wiki/Oculus_Go
498 http://en.wikipedia.org/wiki/VFX1_Headgear
499 http://en.wikipedia.org/wiki/Sega_VR
500 http://en.wikipedia.org/wiki/Pimax_4K
501 http://en.wikipedia.org/wiki/Valve_Index
502 http://en.wikipedia.org/wiki/The_Sword_of_Damocles_(virtual_reality)
503 http://en.wikipedia.org/wiki/Virtual_reality
504 http://en.wikipedia.org/wiki/Violence_and_video_games
505 http://en.wikipedia.org/wiki/Mightier
506 http://en.wikipedia.org/wiki/Axolot_Games
507 http://en.wikipedia.org/wiki/Playbour
508 http://en.wik

651 http://en.wikipedia.org/wiki/Phantom_Breaker
652 http://en.wikipedia.org/wiki/Guilty_Gear_Strive
653 http://en.wikipedia.org/wiki/Dead_or_Alive_5_Last_Round
654 http://en.wikipedia.org/wiki/Guilty_Gear_X2_updated_versions
655 http://en.wikipedia.org/wiki/C64_Direct-to-TV
656 http://en.wikipedia.org/wiki/TV_Tennis_Electrotennis
657 http://en.wikipedia.org/wiki/Ameprod_Television_Game_10
658 http://en.wikipedia.org/wiki/Coleco_Telstar_series
659 http://en.wikipedia.org/wiki/Telejogo_II
660 http://en.wikipedia.org/wiki/Game_&_Watch
661 http://en.wikipedia.org/wiki/Homebrew_(video_games)
662 http://en.wikipedia.org/wiki/Interton_Video_3000
663 http://en.wikipedia.org/wiki/Coleco_Telstar_Marksman
664 http://en.wikipedia.org/wiki/Namco_Museum
665 http://en.wikipedia.org/wiki/Game_&_Watch_series
666 http://en.wikipedia.org/wiki/Ping-O-Tronic
667 http://en.wikipedia.org/wiki/Interton_Video_2501
668 http://en.wikipedia.org/wiki/Interton_Video_3001
669 http://en.wikipedia.org/wiki/PlayStatio

...

2710 http://en.wikipedia.org/wiki/Inel_Tomlinson
2711 http://en.wikipedia.org/wiki/Shroud_(streamer)
2712 http://en.wikipedia.org/wiki/Hafu_(video_game_player)
2713 http://en.wikipedia.org/wiki/YBN_Nahmir
2714 http://en.wikipedia.org/wiki/Ioan-Cristian_Chirilă
2715 http://en.wikipedia.org/wiki/Jacky_Yip
2716 http://en.wikipedia.org/wiki/Tyler1
2717 http://en.wikipedia.org/wiki/Amaz_(video_game_player)
2718 http://en.wikipedia.org/wiki/Offline_TV
2719 http://en.wikipedia.org/wiki/Faker_(gamer)
2720 http://en.wikipedia.org/wiki/Kitboga_(streamer)
2721 http://en.wikipedia.org/wiki/Hai_(gamer)
2722 http://en.wikipedia.org/wiki/Myth_(streamer)
2723 http://en.wikipedia.org/wiki/Bryan_Dechart
2724 http://en.wikipedia.org/wiki/Aui_2000
2725 http://en.wikipedia.org/wiki/Daniel_Naroditsky
2726 http://en.wikipedia.org/wiki/Sasha_Grey
2727 http://en.wikipedia.org/wiki/Mang0
2728 http://en.wikipedia.org/wiki/Lex_Veldhuis
2729 http://en.wikipedia.org/wiki/Etika
2730 http://en.wikipedia.org/wiki/Gegu

2885 http://en.wikipedia.org/wiki/Mumbo_Jumbo_(YouTuber)
2886 http://en.wikipedia.org/wiki/Chills_(entertainer)
2887 http://en.wikipedia.org/wiki/Joseph_Garrett
2888 http://en.wikipedia.org/wiki/Chuggaaconroy
2889 http://en.wikipedia.org/wiki/Jesse_Cox_(entertainer)
2890 http://en.wikipedia.org/wiki/LowTierGod
2891 http://en.wikipedia.org/wiki/Mini_Ladd
2892 http://en.wikipedia.org/wiki/1973_in_video_gaming
2893 http://en.wikipedia.org/wiki/1980_in_video_gaming
2894 http://en.wikipedia.org/wiki/1981_in_video_gaming
2895 http://en.wikipedia.org/wiki/1972_in_video_gaming
2896 http://en.wikipedia.org/wiki/List_of_years_in_video_games
2897 http://en.wikipedia.org/wiki/2004_in_video_gaming
2898 http://en.wikipedia.org/wiki/2015_in_video_gaming
2899 http://en.wikipedia.org/wiki/1996_in_video_gaming
2900 http://en.wikipedia.org/wiki/1971_in_video_gaming
2901 http://en.wikipedia.org/wiki/1976_in_video_gaming
2902 http://en.wikipedia.org/wiki/1986_in_video_gaming
2903 http://en.wikipedia.org/wi

3051 http://en.wikipedia.org/wiki/Steve_Aoki
3052 http://en.wikipedia.org/wiki/Derek_Holland
3053 http://en.wikipedia.org/wiki/Protection
3054 http://en.wikipedia.org/wiki/Zelina_Vega
3055 http://en.wikipedia.org/wiki/YBN_Almighty_Jay
3056 http://en.wikipedia.org/wiki/Evan_Eckenrode
3057 http://en.wikipedia.org/wiki/Anthony_Alfredo
3058 http://en.wikipedia.org/wiki/Evil_Uno
3059 http://en.wikipedia.org/wiki/Benjamin_Lupo
3060 http://en.wikipedia.org/wiki/Dakotaz
3061 http://en.wikipedia.org/wiki/Hafu_(gamer)
3062 http://en.wikipedia.org/wiki/Amber_Wadham
3063 http://en.wikipedia.org/wiki/Anna_Cramling_Bellon
3064 http://en.wikipedia.org/wiki/TheRealSocialDuchess
3065 http://en.wikipedia.org/wiki/Myth_(gamer)
3066 http://en.wikipedia.org/wiki/Phi_Phi_O'Hara
3067 http://en.wikipedia.org/wiki/Antoine_Daniel_(YouTuber)
3068 http://en.wikipedia.org/wiki/Ben_Simmons
3069 http://en.wikipedia.org/wiki/Mike_Shinoda
3070 http://en.wikipedia.org/wiki/Jason_Dolley
3071 http://en.wikipedia.org/wiki

3213 http://en.wikipedia.org/wiki/Persona_5:_Dancing_Star_Night
3214 http://en.wikipedia.org/wiki/Polybius_(2017_video_game)
3215 http://en.wikipedia.org/wiki/PlayStation_VR_Worlds
3216 http://en.wikipedia.org/wiki/List_of_Instant_Game_Collection_games_(PAL_region,_2015)
3217 http://en.wikipedia.org/wiki/List_of_Instant_Game_Collection_games_(PAL_region,_2017)
3218 http://en.wikipedia.org/wiki/List_of_Instant_Game_Collection_games_(PAL_region,_2019)
3219 http://en.wikipedia.org/wiki/Xing:_The_Land_Beyond
3220 http://en.wikipedia.org/wiki/Gang_Beasts
3221 http://en.wikipedia.org/wiki/Tekken_7
3222 http://en.wikipedia.org/wiki/Persona_3:_Dancing_Moon_Night
3223 http://en.wikipedia.org/wiki/Persona_5:_Dancing_in_Starlight
3224 http://en.wikipedia.org/wiki/Golem_(video_game)
3225 http://en.wikipedia.org/wiki/Mini_Motor_Racing
3226 http://en.wikipedia.org/wiki/Hyper_Void
3227 http://en.wikipedia.org/wiki/Don't_Knock_Twice_(video_game)
3228 http://en.wikipedia.org/wiki/Persona_3:_Dancing_in_

3359 http://en.wikipedia.org/wiki/Sky_Captain_and_the_World_of_Tomorrow
3360 http://en.wikipedia.org/wiki/Pixel_Perfect
3361 http://en.wikipedia.org/wiki/Hardwired_(film)
3362 http://en.wikipedia.org/wiki/World_Builder_(film)
3363 http://en.wikipedia.org/wiki/Total_Recall_(1990_film)
3364 http://en.wikipedia.org/wiki/Avatar_(2009_film)
3365 http://en.wikipedia.org/wiki/Paycheck_(film)
3366 http://en.wikipedia.org/wiki/Megaforce
3367 http://en.wikipedia.org/wiki/Mysterio
3368 http://en.wikipedia.org/wiki/Matt_Parkman
3369 http://en.wikipedia.org/wiki/Amanda_Sefton
3370 http://en.wikipedia.org/wiki/Luminus_(comics)
3371 http://en.wikipedia.org/wiki/Chiari_Tsukikage
3372 http://en.wikipedia.org/wiki/Kanon_Mizushiro
3373 http://en.wikipedia.org/wiki/Gypsy_(comics)
3374 http://en.wikipedia.org/wiki/Mastermind_(Jason_Wyngarde)
3375 http://en.wikipedia.org/wiki/Loki_(comics)
3376 http://en.wikipedia.org/wiki/White_Witch
3377 http://en.wikipedia.org/wiki/Duke_of_Deception
3378 http://en.wikipe

In [17]:
#############################################
# Creating tables:
# Unique pages:
#############################################

from bd_vr import BDdatos

db = BDdatos()
con = db.conectar()

otable = 'WikiPagesIDs'

cur = con.cursor()


# Create table:
cur.execute(u"""CREATE TABLE """ + otable + """ (id INT AUTO_INCREMENT PRIMARY KEY)  
SELECT DISTINCT returnedUrl AS wikipageUrl FROM """ + rtable + """;""")

# Drop temporal table:
cur.execute(u"""DROP TABLE """ + ttable + """;""")

db.saveDB(con)
db.closeDB(con)

In [11]:
#La tabla WikiPagesIDs contiene los links de las páginas de la Wikipedia.
